# Testing Connection to Google Sheets

In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import jsmith_acquire
import jsmith_prepare

In [2]:
#Set up credentials
gc = gspread.service_account(filename='credentials.json')

In [3]:
#Open Google Sheet By Name
gsheet = gc.open("Pending Reports")

In [4]:
#Try Loading First Tab, 'DistrictCivil', as a dataframe
original_data = pd.DataFrame(gsheet.sheet1.get_all_records())

In [5]:
original_data.head()

,cause_number,notes,disposed,dispose_date,on_track,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,03-06-19302-MCV,Waiver of service 6/30/2003,,,,,,,,
1,21-12-40648-MCV,WAIVER OF SERVICE 12/09/2021,,,,12/8/2021,DIVORCE - NO CHILDREN,,,FIDEL MARTINEZ JR.
2,07-01-22238-MCV,WAIVER OF SERVICE 01/10/2022,,,,1/9/2007,"MOTION ENFORCEMENT, TO M",5/14/2007,FINAL HEARING,
3,03-08-19404-MCV,Waive 8/11/2003,,,,,,,,
4,15-07-31756-MCV,"UIFSA - FOREIGN SUPPORT ORDER, NOTICE OF REG. ...",FALSE,,,7/27/2015,1,,,"DEHOYOS, MARIO ALBERT"


# Connection Successful!

Now try adding the newly created data from your functions to the other tabs.

In [11]:
#Create a list of PDF paths to iterate through
path_list = ['/Users/johnathonsmith/Downloads/CV.PEND.CASE_Dimmit_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_MAVERICK_2-3-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Maverick_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Dimmit_1-21-2022.pdf']

In [12]:
path_list[2]

'/Users/johnathonsmith/Downloads/CV.PEND.CASE_MAVERICK_2-3-2022.pdf'

In [13]:
#Try adding just one first
df = jsmith_acquire.build_dataframe(path_list[2])
df = jsmith_prepare.prepare_dataframe(df)

Collected Data From 1110 Cases.


In [14]:
df.head()

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Need File,Disposed Date,Finding,Finding Date
0,Maverick,00-02-16392-CV,02/04/2000,(TITLE IV-D OAG USE ONLY,,,,,"ATTORNEY GENERAL\nCHAVARRIA, AZUCENA",MELINDA S. LAUCK,"CHAVARRIA, JOSE M.",,Civil,False,Pending,,,,,
1,Maverick,1-01-33838-MCV,01/11/2017,MOTOR VEHICLE,,,,,"CANO, ANA",ROLANDO SALINAS,"FIDLER, WILLIAM",,Civil,False,Pending,,,,,
2,Maverick,1-03-34076-MCV,03/09/2017,DIVORCE - CHILDREN,,,,,"MORALES, SYLVIA DANIELA",FELIPE HERNANDEZ,"MORALES, ROBERTO",,Civil,False,Pending,,,,,
3,Maverick,01-06-17430-CV,11/09/2017,SHOW CAUSE MOTION,08/29/2001,PETITION TO ESTA,,,"ATTORNEY GENERAL\nMUNOZ, CLARISA",MELINDA S. LAUCK,"BARRIENTOS, JOSE ALEJANDRO",,Civil,False,Pending,,,,,
4,Maverick,1-07-34542-MCV,07/17/2017,DIVORCE - CHILDREN,,,,,"RODRIGUEZ AVALOS, ALEJANDRO",FRANCISCO MARTINEZ III,"AVALOS, MARIELLA",,Civil,False,Pending,,,,,


In [15]:
#Civil cases go to the 'Civil Cases' tab
civil_sheet = gsheet.worksheet('Civil Cases')

#Criminal cases go to the 'Criminal Cases' tab
crim_sheet = gsheet.worksheet('Criminal Cases')

In [16]:
civil_sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Civil Cases'!A1:T1111",
 'updatedRows': 1111,
 'updatedColumns': 20,
 'updatedCells': 22220}

In [17]:
#Create dataframes
civil_df = pd.DataFrame()
crim_df = pd.DataFrame()

In [18]:
#Use a loop to extract all the data and prepare the dataframes accordingly.
#Add them one by one to the google sheet in the proper tabs

for path in path_list:
    #Extract the PDF data
    df = jsmith_acquire.build_dataframe(path)
    
    #Prepare the df and add new columns
    df = jsmith_prepare.prepare_dataframe(df)
    
    if df['Case Type'][0] == 'Criminal':
        #Add to criminal cases tab
        crim_df = crim_df.append(df, ignore_index = True)
    elif df['Case Type'][0] == 'Civil' or df['Case Type'][0] == 'Tax':
        civil_df = civil_df.append(df, ignore_index = True)
    else:
        print('Something went wrong in the loop!')

Collected Data From 644 Cases.
Collected Data From 779 Cases.
Collected Data From 1110 Cases.
Collected Data From 59 Cases.
Collected Data From 221 Cases.
Collected Data From 77 Cases.


In [19]:
#Now add it to the Criminal Cases tab in the Google Sheet
crim_sheet.update([crim_df.columns.values.tolist()] + crim_df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Criminal Cases'!A1:P358",
 'updatedRows': 358,
 'updatedColumns': 16,
 'updatedCells': 5728}

In [20]:
#Next add the data to the Civil Cases tab
civil_sheet.update([civil_df.columns.values.tolist()] + civil_df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Civil Cases'!A1:T2534",
 'updatedRows': 2534,
 'updatedColumns': 20,
 'updatedCells': 50680}